### Battle of the Neighborhoods Denver

#### IBM Data Science capstone

### Introduction/Business Problem
#### Since medical and recreational marijuana have been a reality in Denver, the city has saw a 45% increase in the housing market since 2010. With swarms of people still moving to Denver in present day, my goal here is to check whether the concentration of dispensaries is driving up the cost of living in certain neighborhoods. I have data from the city of Denver on active dispensaries, highest cost of living neighborhoods, and an official list of recognized neighborhoods from the city of Denver.

In [8]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import urllib3

from bs4 import BeautifulSoup

from lxml import html
import lxml.html

import csv
import html5lib

import requests


#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Libraries imported.


### Importing the data for active medical and recreational marijuana dispensaries

In [4]:
active_biz_licenses_data = pd.read_csv('marijuana_active_business_licenses.csv')
active_biz_licenses_data.head()

,Business File Number,License Type,Entity Name,Trade Name,Current License Status,Expiration Date,Facility Street Number,Facility Pre-Direction,Facility Street Name,Facility Street Type,Facility Unit Number,Facility City,Facility Zip Code
0,2017-BFN-0000990,Retail Marijuana Inf Prod Mfg,AMERICAN INDUSTRY LLC,DENVER CLONE STORE,License Issued - Active,10/2/2019 12:00:00 AM,4571,N,Ivy,ST,NaN,Denver,80216.0
1,2010-BFN-1045906,Medical Marijuana Center,"LITTLE BROWN HOUSE DISPENSARY, LLC",LITTLE BROWNHOUSE,License Issued - Active,2/16/2020 12:00:00 AM,1995,S,BROADWAY,NaN,NaN,DENVER,80210.0
2,2010-BFN-1048204,Medical Marijuana Center,"RIVERROCK, LLC",RIVERROCK WELLNESS,License Issued - Active,9/11/2019 12:00:00 AM,4935,N,YORK,ST,NaN,DENVER,80216.0
3,2010-BFN-1048159,Medical Marijuana Center,"DENVER DISPENSARY, LLC",DENVER DISPENSARY,License Issued - Active,9/11/2019 12:00:00 AM,4975,N,VASQUEZ,BLVD,NaN,DENVER,80216.0
4,2012-BFN-1063644,Med Marijuana Opt Prem Cultiv,"DENVER PATIENTS GROUP, LLC",LIVWELL,License Issued - Active,9/16/2019 12:00:00 AM,5141,N,FRANKLIN,ST,NaN,DENVER,80216.0


### Importing the data neighborhood breakdown

In [32]:
denver_neighborhoods = pd.read_csv('statistical_neighborhoods.csv')
denver_neighborhoods.head()

,NBHD_ID,NBHD_NAME,TYPOLOGY,NOTES
0,2,Auraria,None,None
1,21,Cory - Merrill,None,None
2,7,Belcaro,None,None
3,70,Washington Park,None,None
4,71,Washington Park West,None,None


### Pulling data on most expensive neighborhoods in Denver

In [80]:
from urllib.parse import urlencode, urlparse, parse_qs, quote_plus

from lxml.html import fromstring
from requests import get

neighborhood_name = "Cherry Creek"
question = quote_plus("what is the zip code of {}, Denver".format(neighborhood_name))
#raw = get("https://www.google.com/search?q={}".format(question)).text
#attribute_value = "hw:/collection/locations:zip code"

#soupygoodness = BeautifulSoup(raw)
#results = soupygoodness.findAll("div", {"class": "g"}, attribute_value)


url = 'https://google.com/search?q=' + question

response = requests.get(url)

#with open('output.html', 'wb') as f:
#    f.write(response.content)
#webbrowser.open('output.html')
import re
pattern = re.compile(r"8\d{4}")
soup = BeautifulSoup(response.text, 'lxml')
all_zips = []
for g in soup.find_all(class_='g'):
    print(g.text)
    print('-----')
    results = re.findall(pattern, g.text)
    all_zips.extend(results)
    print(results)
    #print([result.match for result in results ])
    
print(sorted(set(all_zips)))



Cherry Creek, Denver, Colorado Zip Code Boundary Map (CO)https://www.zipmap.net/Colorado/Denver.../Z_Cherry_Creek.htmCachedThis page shows a map with an overlay of Zip Codes for Cherry Creek, Denver, Colorado.
 Users can easily view the boundaries of each Zip Code for free.
Cherry Creek, Denver, Colorado Neighborhood Zip Code Polygon Map Version 4.0 Copyright © 1996-2018 USNaviguide LLC.
-----
[]
Best Places to Live in Cherry Creek (zip 80111), Coloradohttps://www.bestplaces.net/zip-code/colorado/cherry_creek/80111CachedCompare Cherry Creek (zip 80111), Colorado to any other place in the USA.
-----
['80111', '80111', '80111']
Cherry Creek Area - Denver - Colorado (CO) - Page 2 - City-Data ...www.city-data.com/forum/denver/23870-cherry-creek-area-2.htmlCachedSimilarNov 13, 2006 ... 80206 is actually Cherry Creek's zip code. Here is where it is going to get confusing though. Cherry Creek is a very upscale neighborhood in central Denver which includes Cherry Creek Shopping Center and the 

In [43]:
source = requests.get('https://www.homesnacks.net/richest-neighborhoods-in-denver-128979/').text
soup = BeautifulSoup(source, 'lxml')

In [44]:
table = soup.find('table')
headers = [th.text for th in table.select("tr th")]
with open("cost_of_living_data.csv", "w") as f:
    wr = csv.writer(f)
    wr.writerow(headers)
    wr.writerows([[td.text.rstrip() for td in row.find_all("td")] for row in table.select("tr")])

In [45]:
cost_of_living_data = pd.read_csv('cost_of_living_data.csv')
cost_of_living_data.head()

,Rank,Neighborhood,Median Household Income
0,1,Cherry Creek,"$112,844"
1,2,Stapleton,"$97,079"
2,3,Washington Park,"$88,971"
3,4,Auraria,"$82,813"
4,5,Central East Denver,"$82,551"


### Converting Denver zipcode data from text to csv

In [25]:
txt_file = r"2018_Gaz_zcta_national.txt"
csv_file = r"zipcodes.csv"

with open(txt_file, 'r') as infile, open(csv_file, 'w') as outfile:
    
     stripped = (line.strip() for line in infile)
     lines = (line.split("\t") for line in stripped if line)
     writer = csv.writer(outfile)
     writer.writerows(lines)

### Cleaning and importing Denver zipcode data 

In [28]:
zipcode_data = pd.read_csv('zipcodes.csv')
zipcode_data = zipcode_data.drop(zipcode_data.columns[[1, 2, 3, 4]], axis=1) 
zipcode_data.rename(columns={'GEOID': 'Zipcode', 'INTPTLAT': 'Latitude', 'INTPTLONG': 'Longitude'}, inplace=True)
zipcode_data.head()

,Zipcode,Latitude,Longitude
0,601,18.180555,-66.749961
1,602,18.361945,-67.175597
2,603,18.455183,-67.119887
3,606,18.158327,-66.932928
4,610,18.295366,-67.125135


### Displaying map of Denver

In [20]:
denver_map = folium.Map(location=[43.6532,-79.3832], zoom_start=11)

for lat, lng, label in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        #parse_html=False
        ).add_to(denver_map)

#display map
denver_map

NameError: name 'df_merged' is not defined